# The Batle of Neighborhoods (Draft)

## Introduction / Business Problem

Foreigners in adjacent Cities (Dhahran and Manama)  in Saudia Arabia & Bharain travel a lot for business purpose and as well as during the weekend. They always look for best quality foods restaurants, shopping stores  and coffee shops. It would be helpful for them to have single and easy interface to look around different venues their ratings, how many are check-ins at anytime etc.

Using the data provided by the Foursquare API of both cities (Dhahran & Manama) a simple & elegant store locator API can be built with customization of local data as well user preferences.

Objective of this study is to visualize the venues and associated details on the map. 

In [58]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Define Foursquare Credentials and Version

In [80]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q5EN3MADVWYFQMAQCDEXFE3SEGAY4QGFV215JDMX4TYOSTAX
CLIENT_SECRET:SYGZZPP5SPS5VNORJP5HRS1OPXKCLX1XIVDYOTALTR30IVH3


#### Let's assume I am staying at 'Le Méridien' hotel and I have two days stay in Easter Province of Saudi Arabia and I want to explore Dhahran City

In [81]:
address = 'Mall Of Dhahran, Saudi Arabia'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

26.30602445 50.1699902281346


#### Let's take a quick look at the complete url and Data.

In [69]:
search_query = "coffee"

In [70]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query,LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Q5EN3MADVWYFQMAQCDEXFE3SEGAY4QGFV215JDMX4TYOSTAX&client_secret=SYGZZPP5SPS5VNORJP5HRS1OPXKCLX1XIVDYOTALTR30IVH3&ll=26.30602445,50.1699902281346&v=20180604&query=coffee&limit=30'

In [71]:
results = requests.get(url).json()
#results

In [72]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
df = dataframe

In [73]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

In [74]:
# function that extracts the category of the venue
# retrieve the name from nested json of categories,
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name'] #it is the key line that checks array each record with 'name' column


In [75]:
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)


In [76]:
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]


In [77]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

In [79]:
# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Le Méridien',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map